In [ ]:
import pandas as pd

jeopardy = pd.read_csv('jeopardy.csv')
#print(jeopardy.head())
print(jeopardy.columns)
jeopardy.columns = ['Show Number', 'Air Date', 'Round', 'Category', 'Value', 'Question', 'Answer']
print(jeopardy.columns)

import re

def norm(string):
    text = string.lower()
    text = re.sub('[^A-Za-z0-9\s]','',text)
    return text

jeopardy['clean_question'] = jeopardy.Question.apply(norm)
jeopardy['clean_answer'] = jeopardy.Answer.apply(norm)

#print(jeopardy.head())


def norm_value(text):
    text = re.sub('[^A-Za-z0-9\s]','',text)
    try:
        num = int(text)
    except Exception:
        num = 0
    return num

jeopardy['clean_value'] = jeopardy.Value.apply(norm_value)
jeopardy['Air Date'] = pd.to_datetime(jeopardy['Air Date'])

jeopardy



def repeat(row):
    split_answer = row['clean_answer'].split(' ')
    split_question = row['clean_question'].split(' ')
    if 'the' in split_answer:
        split_answer.remove('the')
    match_count = 0
    if len(split_answer) == 0:
        return 0
    for i in split_answer:
        if i in split_question:
            match_count += 1
    return match_count/len(split_answer)

jeopardy['answer_in_question'] = jeopardy.apply(repeat, axis = 1)
jeopardy['answer_in_question'].mean()




question_overlap = []
terms_used = set()
for i, row in jeopardy.iterrows():
        split_question = row["clean_question"].split(" ")
        split_question = [q for q in split_question if len(q) > 5]
        match_count = 0
        for word in split_question:
            if word in terms_used:
                match_count += 1
        for word in split_question:
            terms_used.add(word)
        if len(split_question) > 0:
            match_count /= len(split_question)
        question_overlap.append(match_count)
jeopardy["question_overlap"] = question_overlap

jeopardy["question_overlap"].mean()





def highvalue(row):
    if row['clean_value'] > 800:
        value = 1
    else:
        value = 0 
    return value
jeopardy['high_value'] = jeopardy.apply(highvalue, axis = 1)

def count_usage(term):
    low_count = 0
    high_count = 0
    for i, row in jeopardy.iterrows():
        if term in row["clean_question"].split(" "):
            if row["high_value"] == 1:
                high_count += 1
            else:
                low_count += 1
    return high_count, low_count

comparison_terms = list(terms_used)[:5]
observed_expected = []
for term in comparison_terms:
    observed_expected.append(count_usage(term))

observed_expected



from scipy.stats import chisquare
import numpy as np

high_value_count = jeopardy[jeopardy.high_value == 1].shape[0]
low_value_count = jeopardy[jeopardy.high_value == 0].shape[0]
chi_squared =[]

for i in observed_expected:
    total = sum(i)
    total_prop = total/jeopardy.shape[0]
    
